## C03 Making Fast Function Calls
### Using Globals
> Don't use globals in performance critical parts of code

#### The trouble with globals
Compiler has been unable to infer the type of result when working with the global variable, marking it as `Any`

In [1]:
using BenchmarkTools

In [2]:
p = 2
function pow_array(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p
    end
    return s
end

pow_array (generic function with 1 method)

In [3]:
t = rand(100000);
@btime pow_array(t);

  3.891 ms (300000 allocations: 4.58 MiB)


In [4]:
@code_warntype pow_array(t)

MethodInstance for pow_array(::Vector{Float64})
  from pow_array(x::Vector{Float64}) @ Main ~/code/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#

::Core.Const(pow_array)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Any
  y::Float64
Body::Any
1 ─

(s = 0.0)


│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%

2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Any
│   %11 = (y ^ Main.p)::Any
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



#### Fixing performance issues with globals
There are two ways to correct the issues of globals
1. `const p`: which will constrain the type of variable p.
2. `pow(array::Vector{Float64}, p)`: make the `p` being the argument of function, during which the type of `p` will be inference.

In [5]:
const p2 = 2
function pow_array2(x::Vector{Float64})
    s = 0.0
    for y in x
        s += y^p2
    end
    return s
end

pow_array2 (generic function with 1 method)

In [6]:
@btime pow_array2(t);

  440.912 μs (0 allocations: 0 bytes)


In [7]:
@code_warntype pow_array2(t)

MethodInstance for pow_array2(::Vector{Float64})
  from pow_array2(x::Vector{Float64}) @ Main ~/code/Julia-HPC/03-Functions.ipynb:2
Arguments
  #self#::Core.Const(pow_array2)
  x::Vector{Float64}
Locals
  @_3::Union{Nothing, Tuple{Float64, Int64}}
  s::Float64
  y::Float64
Body::Float64
1 ─       (s = 0.0)
│   %2  = x::Vector{Float64}
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Float64, Int64}
│         (y = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = s::Float64
│   %11 = (y ^ Main.p2)::Float64
│         (s = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s



In [8]:
function pow_array3(x::Vector{Float64})
    return pow_array_inner(x, p)
end

function pow_array_inner(x, pow)
    s = 0.0
    for y in x
        s += y^pow
    end
    return s
end

pow_array_inner (generic function with 1 method)

In [9]:
@btime pow_array3(t);

  313.248 μs (1 allocation: 16 bytes)


### Inlining
#### Default Inlining


In [10]:
function f(x)
    a = x * 5
    b = a + 3
end
g(x) = f(2 * x)

g (generic function with 1 method)

In [14]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = Base.mul_int(%1, 5)::Int64
│   %3 = Base.add_int(%2, 3)::Int64
└──      return %3
) => Int64

In [15]:
@code_llvm g(3)

;  @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:5 within `g`
define i64 @julia_g_1771(i64 signext %0) #0 {
top:
; ┌ @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:2 within `f`
; │┌ @ int.jl:88 within `*`
    %1 

= mul i64 %0, 10
; │└
; │ @ /home/zpp/code/Julia-HPC/03-Functions.ipynb:3 within `f`
; │┌ @ int.jl:87 within `+`
    %2 = add i64 %1, 3
; └└
  ret i64 %2
}


#### Controlling inlining
- `@inline`: make function be inlining
- `@noinline`: make function not be inlining

In [18]:
@noinline function f(x)
    a = x * 5
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end


f (generic function with 1 method)

In [19]:
@code_typed g(3)

CodeInfo(
1 ─ %1 = Base.mul_int(2, x)::Int64
│   %2 = invoke Main.f(%1::Int64)::Int64
└──      return %2
) => Int64

In [23]:
@inline function f_in(x)
    a = x * 5
    b = a + 3
    c = a - 4
    if c < 0
        throw(DomainError())
    elseif c < 2
        d = c^3
    else
        d = c^2
    end
end

g_in(x) = f_in(2 * x)

g_in (generic function with 1 method)

In [24]:
@code_typed g_in(3)

CodeInfo(
1 ─ %1  = Base.mul_int(2, x)::Int64
│   %2  = Base.mul_int(%1, 5)::Int64
│   %3  = Base.sub_int(%2, 4)::Int64
│   %4  = Base.slt_int(%3, 0)::Bool
└──       goto #3 if not %4
2 ─       Main.DomainError()::Union{}
└──       unreachable
3 ─ %8  = Base.slt_int(%3, 2)::Bool
└──       goto #5 if not %8
4 ─ %10 = Base.mul_int(%3, %3)::Int64
│   %11 = Base.mul_int(%10, %3)::Int64
└──       goto #6
5 ─ %13 = Base.mul_int(%3, %3)::Int64
└──       goto #6
6 ┄ %15 = φ (#4 => %11, #5 => %13)::Int64
└──       return %15
) => Int64

In [39]:
@btime g(3);
@btime g_in(3);

  1.264 ns (0 allocations: 0 bytes)


  1.263 ns (0 allocations: 0 bytes)


### Constant propagation


In [47]:
sqr(x) = x * x;
sqr2() = sqr(2)

sqr2 (generic function with 1 method)

In [46]:
@code_typed sqr(2)

CodeInfo(
1 ─ %1 = Base.mul_int(x, x)::Int64
└──      return %1
) => Int64

In [41]:
@code_typed sqr2()

CodeInfo(
1 ─     return 4
) => Int64